# Fie Upon Thee, Autocorrect!

<img src="img/shakespeare.jpg" width="200">

<br><br><br>

## Recap of yesterday

<br><br><br>

## Dataset: the complete works of Shakespeare

In [1]:
import numpy as np

In [2]:
with open("data/shakespeare.txt") as file:
    corpus = file.read()

In [3]:
len(corpus)

5367504

In [4]:
print(corpus[100000:101000])

ook up that fire,
Which many legions of true hearts had warmed,
And so the general of hot desire,
Was sleeping by a virgin hand disarmed.
This brand she quenched in a cool well by,
Which from Love’s fire took heat perpetual,
Growing a bath and healthful remedy,
For men diseased; but I, my mistress’ thrall,
  Came there for cure and this by that I prove,
  Love’s fire heats water, water cools not love.

THE END

ALL’S WELL THAT ENDS WELL

Contents

ACT I
Scene I. Rossillon. A room in the Countess’s palace.
Scene II. Paris. A room in the King’s palace.
Scene III. Rossillon. A Room in the Palace.

ACT II
Scene I. Paris. A room in the King’s palace.
Scene II. Rossillon. A room in the Countess’s palace.
Scene III. Paris. The King’s palace.
Scene IV. Paris. The King’s palace.
Scene V. Another room in the same.

ACT III
Scene I. Florence. A room in the Duke’s palace.
Scene II. Rossillon. A room in the Countess’s palace.
Scene III. Florence. Before the Duke’s palace.
Scene IV. Rossillon. A roo

In [5]:
unique_characters = np.unique(np.frombuffer(corpus.encode(), np.uint8).reshape(-1, 1))
unique_characters

array([ 10,  32,  33,  35,  36,  37,  38,  40,  41,  42,  44,  45,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
        63,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  93,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106,
       107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
       120, 121, 122, 128, 132, 134, 135, 137, 147, 148, 152, 153, 156,
       157, 160, 162, 166, 167, 168, 169, 170, 171, 174, 187, 191, 195,
       197, 226, 239], dtype=uint8)

In [6]:
unique_characters.view("S1")

array([b'\n', b' ', b'!', b'#', b'$', b'%', b'&', b'(', b')', b'*', b',',
       b'-', b'.', b'/', b'0', b'1', b'2', b'3', b'4', b'5', b'6', b'7',
       b'8', b'9', b':', b';', b'?', b'A', b'B', b'C', b'D', b'E', b'F',
       b'G', b'H', b'I', b'J', b'K', b'L', b'M', b'N', b'O', b'P', b'Q',
       b'R', b'S', b'T', b'U', b'V', b'W', b'X', b'Y', b'Z', b'[', b']',
       b'a', b'b', b'c', b'd', b'e', b'f', b'g', b'h', b'i', b'j', b'k',
       b'l', b'm', b'n', b'o', b'p', b'q', b'r', b's', b't', b'u', b'v',
       b'w', b'x', b'y', b'z', b'\x80', b'\x84', b'\x86', b'\x87',
       b'\x89', b'\x93', b'\x94', b'\x98', b'\x99', b'\x9c', b'\x9d',
       b'\xa0', b'\xa2', b'\xa6', b'\xa7', b'\xa8', b'\xa9', b'\xaa',
       b'\xab', b'\xae', b'\xbb', b'\xbf', b'\xc3', b'\xc5', b'\xe2',
       b'\xef'], dtype='|S1')

In [7]:
np.count_nonzero(unique_characters <= ord("z"))

81

<br><br><br>

## In progress...

In [419]:
import re

In [448]:
words = set()
for match in re.finditer(r"\b[A-Za-z]{1,10}\b", corpus):
    words.add(match.group(0).upper())

In [585]:
def word_to_vector(word):
    vector = np.zeros((10, 27))
    for j, letter in enumerate(word):
        vector[j, ord(letter) - ord("A") + 1] = 1
    for j2 in range(j + 1, 10):
        vector[j2, 0] = 1
    return vector

In [587]:
vectors = np.zeros((len(words), 10, 27))
for i, word in enumerate(words):
    vectors[i] = word_to_vector(word)

In [588]:
def vector_to_word(vector):
    return (np.argmax(vector, axis=1) + ord("A") - 1).astype(np.uint8).tobytes().decode().rstrip("@")

In [589]:
vector_to_word(vectors[7])

'GRASS'

In [590]:
vectors.shape

(22803, 10, 27)

In [748]:
neural_network = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=(230, 180, 120, 80, 50, 80, 120, 180, 230))

In [749]:
neural_network.fit(vectors.reshape(-1, 270), vectors.reshape(-1, 270))

MLPRegressor(hidden_layer_sizes=(230, 180, 120, 80, 50, 80, 120, 180, 230))

In [750]:
for vector in vectors[:30]:
    predicted_vector = neural_network.predict(vector.reshape(1, 270)).reshape(10, 27)
    print(f"{vector_to_word(vector) = :13.13s} {vector_to_word(predicted_vector) = :s}")

vector_to_word(vector) = AMAZETH       vector_to_word(predicted_vector) = ATADETS
vector_to_word(vector) = PUR           vector_to_word(predicted_vector) = PUR
vector_to_word(vector) = SCHOOLING     vector_to_word(predicted_vector) = STPOOLING
vector_to_word(vector) = INFLUENCE     vector_to_word(predicted_vector) = INULLENSE
vector_to_word(vector) = DIETED        vector_to_word(predicted_vector) = DIETED
vector_to_word(vector) = PESEECH       vector_to_word(predicted_vector) = PESEETD
vector_to_word(vector) = SOD           vector_to_word(predicted_vector) = SO
vector_to_word(vector) = GRASS         vector_to_word(predicted_vector) = GRASS
vector_to_word(vector) = CASH          vector_to_word(predicted_vector) = CASH
vector_to_word(vector) = IDLY          vector_to_word(predicted_vector) = ITLU
vector_to_word(vector) = REVISIT       vector_to_word(predicted_vector) = REVISIT
vector_to_word(vector) = HUMBLER       vector_to_word(predicted_vector) = HUMPLER
vector_to_word(vector) = PROCE

In [751]:
first_half_network = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=(230, 180, 120, 80))
first_half_network.__sklearn_is_fitted__ = lambda: True
first_half_network.n_layers_ = 6
first_half_network.coefs_ = neural_network.coefs_[:5]
first_half_network.intercepts_ = neural_network.intercepts_[:5]
first_half_network.out_activation_ = "relu"

In [752]:
second_half_network = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=(80, 120, 180, 230))
second_half_network.__sklearn_is_fitted__ = lambda: True
second_half_network.n_layers_ = 6
second_half_network.coefs_ = neural_network.coefs_[5:]
second_half_network.intercepts_ = neural_network.intercepts_[5:]
second_half_network.out_activation_ = "relu"

In [753]:
encoded = first_half_network.predict(vectors[7].reshape(1, 270))
encoded

array([[1.34543555, 0.64810183, 2.49781079, 0.        , 0.        ,
        0.57868294, 0.        , 0.        , 0.        , 2.24002717,
        0.        , 1.1172438 , 2.95279067, 0.        , 0.        ,
        0.        , 1.03808674, 4.57588084, 2.93460316, 0.5413308 ,
        1.10784638, 2.10569659, 0.        , 2.74750654, 0.        ,
        2.58979996, 2.17716639, 0.        , 0.68081638, 1.99053661,
        2.0229948 , 0.58634946, 0.        , 1.72725972, 1.54810959,
        1.00517415, 0.83060548, 0.        , 1.73232823, 1.2975571 ,
        1.99654483, 0.        , 1.50701804, 0.        , 3.04116681,
        0.97869101, 0.67561146, 1.50237863, 0.        , 1.07072389]])

In [754]:
decoded = second_half_network.predict(encoded).reshape(10, 27)
vector_to_word(decoded)

'GRASS'

In [755]:
distribution = first_half_network.predict(vectors.reshape(-1, 270))

In [756]:
center = np.mean(distribution, axis=0)

In [757]:
width = np.std(distribution, axis=0)

In [788]:
fake_encoded = np.random.normal(center, width, (1, 50,))
fake_decoded = second_half_network.predict(fake_encoded).reshape(10, 27)
vector_to_word(fake_decoded)

'TECSNLT'